In [ ]:
import os, glob
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller, kpss

# -------- config ----------
folder_before = r"D:/NCKH/vnecon_before_scandals"
out_dir = os.path.join(folder_before, "_results_before")
os.makedirs(out_dir, exist_ok=True)

# Ngưỡng nhận diện chuỗi (gần) hằng số
TOL_CONST = 1e-12

# -------- utils ----------
def zscore(df: pd.DataFrame) -> pd.DataFrame:
    return (df - df.mean()) / df.std(ddof=0)

def pick_lag(endog: pd.DataFrame, maxcaps: int = 4) -> int:
    n = len(endog)
    maxlags = min(maxcaps, max(1, n // 25))  # n≈200 -> max 4
    try:
        sel = VAR(endog).select_order(maxlags)
        lag = sel.selected_orders.get("aic") or 1
    except Exception:
        lag = 1
    return int(max(1, min(lag, maxlags)))

def fit_var(endog: pd.DataFrame):
    # bỏ regime nếu biến gần hằng số
    if (endog.var() < 1e-10).any():
        raise ValueError("Near-constant series; skip.")
    p = pick_lag(endog)
    try:
        return VAR(endog).fit(p)
    except np.linalg.LinAlgError:
        # jitter phá suy biến
        jitter = endog + np.random.normal(scale=1e-6, size=endog.shape)
        return VAR(jitter).fit(p)

def _is_constant(x: pd.Series, tol=TOL_CONST) -> bool:
    x = pd.Series(x).dropna()
    if len(x) == 0:
        return True
    return float(x.var()) < tol or x.nunique() <= 1

def adf_test(x, regression: str = "c"):
    """Augmented Dickey-Fuller: H0 = có unit root (không dừng)."""
    x = pd.Series(x).dropna()
    # Nếu chuỗi hằng số, adfuller sẽ lỗi -> coi như dừng: ADF p=0
    if _is_constant(x):
        return {"stat": np.nan, "pvalue": 0.0, "lags": 0, "nobs": len(x)}
    if len(x) < 10:
        # quá ngắn: trả p lớn để buộc diff tiếp
        return {"stat": np.nan, "pvalue": 1.0, "lags": np.nan, "nobs": len(x)}
    res = adfuller(x, regression=regression, autolag="AIC")
    return {"stat": res[0], "pvalue": res[1], "lags": res[2], "nobs": res[3]}

def kpss_test(x, regression: str = "c", nlags: str = "auto"):
    """KPSS: H0 = dừng (level/trend-stationary)."""
    x = pd.Series(x).dropna()
    # Nếu chuỗi hằng số, coi như dừng: KPSS p=1
    if _is_constant(x):
        return {"stat": np.nan, "pvalue": 1.0, "lags": 0}
    if len(x) < 10:
        # quá ngắn: trả p nhỏ để ép diff tiếp
        return {"stat": np.nan, "pvalue": 0.0, "lags": np.nan}
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        try:
            stat, pval, lags, _ = kpss(x, regression=regression, nlags=nlags)
        except Exception:
            # khi chuỗi quá 'bất thường', trả về p nhỏ để buộc diff tiếp
            stat, pval, lags = np.nan, 0.0, np.nan
    return {"stat": stat, "pvalue": pval, "lags": lags}

def stationarity_report(df: pd.DataFrame, level_reg: str = "c") -> pd.DataFrame:
    """
    In ra ADF (muốn bác bỏ H0) và KPSS (không muốn bác bỏ H0).
    level_reg: 'c' cho level-stationary; nếu dữ liệu có xu hướng, dùng 'ct'.
    """
    print("\n[Stationarity checks]")
    rows = []
    for col in df.columns:
        a = adf_test(df[col], regression=level_reg)
        k = kpss_test(df[col], regression=level_reg)
        rows.append({
            "var": col,
            "ADF_p": a["pvalue"],
            "KPSS_p": k["pvalue"],
            "ADF_stat": a["stat"],
            "KPSS_stat": k["stat"],
            "ADF_lags": a["lags"],
            "KPSS_lags": k["lags"],
            "nobs": a.get("nobs", len(df[col].dropna()))
        })
    rep = pd.DataFrame(rows)
    print(rep.to_string(index=False))
    return rep

def make_stationary(
    df: pd.DataFrame, max_d: int = 2, level_reg: str = "c", alpha: float = 0.05
):
    """
    Tự động difference từng biến tối đa max_d lần cho tới khi:
      - ADF p < alpha (bác bỏ unit root), và
      - KPSS p >= alpha (không bác bỏ dừng)
    Trả về:
      df_out: DataFrame sau khi làm dừng
      diffs:  dict {col: số lần diff}
      report: bảng kết quả ADF/KPSS cuối cùng
    """
    df_out = df.copy()
    diffs = {c: 0 for c in df.columns}

    for col in df.columns:
        series = df_out[col].copy()
        d = 0
        while d <= max_d:
            a = adf_test(series, regression=level_reg)
            k = kpss_test(series, regression=level_reg)
            # Điều kiện “ổn”: ADF bác bỏ H0 (p<alpha) & KPSS không bác bỏ H0 (p>=alpha)
            if (a["pvalue"] < alpha) and (k["pvalue"] >= alpha):
                break
            if d == max_d:
                print(f"⚠️  {col}: chưa đạt cả ADF&KPSS sau {max_d} lần diff; dùng D^{d}{col}.")
                break
            # diff thêm
            series = series.diff().dropna()
            d += 1

        df_out[col] = series
        diffs[col] = d

    # align lại vì các cột diff khác bậc sẽ lệch chỉ mục
    df_out = df_out.dropna().copy()

    print("\n[Applied differences]")
    for c, d in diffs.items():
        print(f"  {c}: diff {d} time(s)")

    report = stationarity_report(df_out, level_reg=level_reg)
    return df_out, diffs, report

def plot_irf_save(irf, title: str, save_path: str, orth: bool = False):
    obj = irf.plot(orth=orth)  # có thể trả Axes hoặc Figure tùy phiên bản
    fig = obj if hasattr(obj, "savefig") else plt.gcf()
    plt.suptitle(title)
    fig.savefig(save_path, dpi=150, bbox_inches="tight")
    plt.close(fig)

# -------- core pipeline ----------
def run_before(path: str, ticker: str):
    df = pd.read_excel(path).sort_values("date")

    # return & continuous score
    df["close"] = pd.to_numeric(df["close"], errors="coerce")
    # = ln(C_t) - ln(C_{t-1}) = ln(C_t / C_{t-1})
    df["ret"] = np.log(df["close"].replace(0, np.nan)).diff()

    # score = tích cực - tiêu cực
    df["score"] = (
        pd.to_numeric(df.get("Tích cực"), errors="coerce")
        - pd.to_numeric(df.get("Tiêu cực"), errors="coerce")
    )

    data = df[["ret", "score"]].replace([np.inf, -np.inf], np.nan).dropna()
    if len(data) < 30:
        print(f"{ticker}: observations too small ({len(data)}). Skipped.")
        return

    # Nếu có biến gần hằng số → bỏ qua ticker (VAR/IRF sẽ không có ý nghĩa)
    if (data.var() < TOL_CONST).any():
        bad_cols = [c for c in data.columns if float(data[c].var()) < TOL_CONST]
        print(f"{ticker}: near-constant series in {bad_cols}. Skipped.")
        return

    # --- Kiểm định & làm dừng nếu cần ---
    # Với return (ret) thường đã ~I(0) nhưng vẫn kiểm định cho chắc; score thì dễ I(1).
    # level_reg='c' (constant). Nếu thấy xu hướng rõ rệt, thử đổi 'ct'.
    data_stationary, diffs, rep = make_stationary(
        data, max_d=2, level_reg="c", alpha=0.05
    )

    if len(data_stationary) < 30:
        print(f"{ticker}: observations too small before differencing ({len(data_stationary)}). Skipped.")
        return

    # Z-score & chia regime theo median của score (tại t-1)
    Z = zscore(data_stationary)
    thr = Z["score"].median()
    low = Z[Z["score"].shift(1) <= thr].dropna().reset_index(drop=True)
    high = Z[Z["score"].shift(1) > thr].dropna().reset_index(drop=True)

    print(f"\n=== {ticker} (before) ===")
    print(f"Obs Low: {len(low)} | Obs High: {len(high)} | Threshold≈{thr: .3f}")

    # fit từng regime
    res_low = res_high = None
    try:
        res_low = fit_var(low)
        print("\n[Low regime] lag =", res_low.k_ar)
        print(res_low.summary())
    except Exception as e:
        print("Low regime failed:", e)

    try:
        res_high = fit_var(high)
        print("\n[High regime] lag =", res_high.k_ar)
        print(res_high.summary())
    except Exception as e:
        print("High regime failed:", e)

    # IRF (orth=False để không cần Cholesky PD)
    if res_low is not None:
        try:
            irf_l = res_low.irf(15)
            plot_irf_save(
                irf_l,
                f"IRF - {ticker} before - Low Sentiment",
                os.path.join(out_dir, f"{ticker}_before_IRF_low.png"),
                orth=False
            )
        except Exception as e:
            print("IRF Low failed:", e)

    if res_high is not None:
        try:
            irf_h = res_high.irf(15)
            plot_irf_save(
                irf_h,
                f"IRF - {ticker} before - High Sentiment",
                os.path.join(out_dir, f"{ticker}_before_IRF_high.png"),
                orth=False
            )
        except Exception as e:
            print("IRF High failed:", e)

# -------- run all before files ----------
if __name__ == "__main__":
    files = glob.glob(os.path.join(folder_before, "*.xlsx"))
    if not files:
        print("Không thấy .xlsx trong:", folder_before)

    for f in files:
        ticker = os.path.splitext(os.path.basename(f))[0]
        run_before(f, ticker)



[Applied differences]
  ret: diff 0 time(s)
  score: diff 1 time(s)

[Stationarity checks]
  var        ADF_p  KPSS_p   ADF_stat  KPSS_stat  ADF_lags  KPSS_lags  nobs
  ret 1.170759e-25     0.1 -13.724552   0.303931         1          3   422
score 2.234800e-12     0.1  -8.008439   0.128368        17         94   406

=== AMD (AFTER) ===
Obs Low: 212 | Obs High: 211 | Threshold≈-0.024

[Low regime] lag = 1
  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 01, Oct, 2025
Time:                     22:48:57
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                 -0.0663817
Nobs:                     211.000    HQIC:                 -0.123168
Log likelihood:          -575.733    FPE:                   0.850704
AIC:                    -0.161695    Det(Omega_mle):        0.827019
--------------------------------------------------------------------


In [3]:
import os, glob
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.stats.diagnostic import acorr_ljungbox

# ===== config =====
folder_before = r"D:/NCKH/vnecon_before_scandals"
out_dir = os.path.join(folder_before, "_results_before")
os.makedirs(out_dir, exist_ok=True)

# Seed để tái lập (đặc biệt khi dùng jitter)
np.random.seed(42)

# Ngưỡng nhận diện chuỗi (gần) hằng số — dùng thống nhất khắp nơi
TOL_CONST = 1e-10

# ===== utils =====
def zscore(df: pd.DataFrame) -> pd.DataFrame:
    return (df - df.mean()) / df.std(ddof=0)

def pick_lag(endog: pd.DataFrame, maxcaps: int = 8) -> int:
    """
    Chọn bậc trễ tối ưu theo BIC (fallback AIC), với trần mềm tay hơn:
    p_max = min(maxcaps, sqrt(n)). Thực tế giúp tránh p=1 quá thường xuyên.
    """
    n = len(endog)
    heuristic = max(1, int(np.sqrt(n)))
    maxlags = min(maxcaps, heuristic)
    try:
        sel = VAR(endog).select_order(maxlags)
        lag = sel.selected_orders.get("bic") or sel.selected_orders.get("aic") or 1
    except Exception:
        lag = 1
    return int(max(1, min(lag, maxlags)))

def fit_var(endog: pd.DataFrame):
    # Bỏ nếu có biến gần hằng số
    if (endog.var() < TOL_CONST).any():
        raise ValueError("Near-constant series; skip.")
    p = pick_lag(endog)
    try:
        return VAR(endog).fit(p)
    except np.linalg.LinAlgError:
        # jitter phá suy biến
        jitter = endog + np.random.normal(scale=1e-6, size=endog.shape)
        return VAR(jitter).fit(p)

def _is_constant(x: pd.Series, tol=TOL_CONST) -> bool:
    x = pd.Series(x).dropna()
    if len(x) == 0:
        return True
    return float(x.var()) < tol or x.nunique() <= 1

def adf_test(x, regression: str = "c"):
    """ADF: H0 = có unit root (không dừng)."""
    x = pd.Series(x).dropna()
    if _is_constant(x):
        # Chuỗi hằng số — coi như dừng: ADF p=0
        return {"stat": np.nan, "pvalue": 0.0, "lags": 0, "nobs": len(x)}
    if len(x) < 10:
        return {"stat": np.nan, "pvalue": 1.0, "lags": np.nan, "nobs": len(x)}
    res = adfuller(x, regression=regression, autolag="AIC")
    return {"stat": res[0], "pvalue": res[1], "lags": res[2], "nobs": res[3]}

def kpss_test(x, regression: str = "c", nlags: str = "auto"):
    """KPSS: H0 = dừng (level/trend-stationary)."""
    x = pd.Series(x).dropna()
    if _is_constant(x):
        # Chuỗi hằng số — coi như dừng: KPSS p=1
        return {"stat": np.nan, "pvalue": 1.0, "lags": 0}
    if len(x) < 10:
        return {"stat": np.nan, "pvalue": 0.0, "lags": np.nan}
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        try:
            stat, pval, lags, _ = kpss(x, regression=regression, nlags=nlags)
        except Exception:
            stat, pval, lags = np.nan, 0.0, np.nan
    return {"stat": stat, "pvalue": pval, "lags": lags}

def stationarity_report(df: pd.DataFrame, level_reg: str = "c") -> pd.DataFrame:
    """In bảng ADF/KPSS cho từng biến."""
    print("\n[Stationarity checks]")
    rows = []
    for col in df.columns:
        a = adf_test(df[col], regression=level_reg)
        k = kpss_test(df[col], regression=level_reg)
        rows.append({
            "var": col,
            "ADF_p": a["pvalue"],
            "KPSS_p": k["pvalue"],
            "ADF_stat": a["stat"],
            "KPSS_stat": k["stat"],
            "ADF_lags": a["lags"],
            "KPSS_lags": k["lags"],
            "nobs": a.get("nobs", len(df[col].dropna()))
        })
    rep = pd.DataFrame(rows)
    print(rep.to_string(index=False))
    return rep

def make_stationary(
    df: pd.DataFrame, max_d: int = 2, level_reg: str = "c",
    alpha: float = 0.05, alpha_kpss: float | None = None
):
    """
    Difference từng biến tối đa max_d lần cho tới khi:
      - ADF p < alpha, và
      - KPSS p >= alpha_kpss (mặc định = alpha; có thể set 0.06 để an toàn biên).
    """
    if alpha_kpss is None:
        alpha_kpss = alpha

    df_out = df.copy()
    diffs = {c: 0 for c in df.columns}

    for col in df.columns:
        series = df_out[col].copy()
        d = 0
        while d <= max_d:
            a = adf_test(series, regression=level_reg)
            k = kpss_test(series, regression=level_reg)
            if (a["pvalue"] < alpha) and (k["pvalue"] >= alpha_kpss):
                break
            if d == max_d:
                print(f"⚠️  {col}: chưa đạt ADF&KPSS sau {max_d} lần diff; dùng D^{d}{col}.")
                break
            series = series.diff().dropna()
            d += 1

        df_out[col] = series
        diffs[col] = d

    df_out = df_out.dropna().copy()

    print("\n[Applied differences]")
    for c, d in diffs.items():
        print(f"  {c}: diff {d} time(s)")

    report = stationarity_report(df_out, level_reg=level_reg)
    return df_out, diffs, report

def plot_irf_save(irf, title: str, save_path: str, orth: bool = False):
    obj = irf.plot(orth=orth)
    fig = obj if hasattr(obj, "savefig") else plt.gcf()
    plt.suptitle(title)
    fig.savefig(save_path, dpi=150, bbox_inches="tight")
    plt.close(fig)

def diag_var(label: str, res):
    """Chẩn đoán ổn định & tự tương quan phần dư (tùy chọn)."""
    try:
        stable = res.is_stable()
        max_root = np.max(np.abs(res.roots))
        print(f"\n[Diag {label}] Stable? {stable} | Max root = {max_root:.3f}")
        for col in res.resid.columns:
            lb = acorr_ljungbox(res.resid[col], lags=[10], return_df=True)
            p10 = float(lb["lb_pvalue"].iloc[0])
            print(f"Ljung-Box({col}, lag10) p = {p10:.3f}")
    except Exception as e:
        print(f"Diag failed ({label}):", e)

# ===== core pipeline =====
def run_before(path: str, ticker: str):
    # Đọc Excel robust hơn
    df = pd.read_excel(path, engine="openpyxl")
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")
        df = df.sort_values("date")
    else:
        df = df.sort_index()

    # return & continuous score
    df["close"] = pd.to_numeric(df["close"], errors="coerce")
    df["ret"] = np.log(df["close"].replace(0, np.nan)).diff()

    # score = tích cực - tiêu cực
    df["score"] = (
        pd.to_numeric(df.get("Tích cực"), errors="coerce")
        - pd.to_numeric(df.get("Tiêu cực"), errors="coerce")
    )

    data = df[["ret", "score"]].replace([np.inf, -np.inf], np.nan).dropna()
    if len(data) < 30:
        print(f"{ticker}: observations too small ({len(data)}). Skipped.")
        return

    # Loại nếu có biến gần hằng số
    if (data.var() < TOL_CONST).any():
        bad_cols = [c for c in data.columns if float(data[c].var()) < TOL_CONST]
        print(f"{ticker}: near-constant series in {bad_cols}. Skipped.")
        return

    # Kiểm định & làm dừng
    data_stationary, diffs, rep = make_stationary(
        data, max_d=2, level_reg="c", alpha=0.05, alpha_kpss=0.06
    )
    if len(data_stationary) < 30:
        print(f"{ticker}: observations too small after differencing ({len(data_stationary)}). Skipped.")
        return

    # Z-score & chia regime theo median(score) tại t-1
    Z = zscore(data_stationary)
    thr = Z["score"].median()
    low  = Z[Z["score"].shift(1) <= thr].dropna().reset_index(drop=True)
    high = Z[Z["score"].shift(1) >  thr].dropna().reset_index(drop=True)

    print(f"\n=== {ticker} (BEFORE) ===")
    print(f"Obs Low: {len(low)} | Obs High: {len(high)} | Threshold≈{thr: .3f}")

    # Fit từng regime
    res_low = res_high = None
    try:
        res_low = fit_var(low)
        print("\n[Low regime] lag =", res_low.k_ar)
        print(res_low.summary())
        diag_var("LOW", res_low)
    except Exception as e:
        print("Low regime failed:", e)

    try:
        res_high = fit_var(high)
        print("\n[High regime] lag =", res_high.k_ar)
        print(res_high.summary())
        diag_var("HIGH", res_high)
    except Exception as e:
        print("High regime failed:", e)

    # IRF
    if res_low is not None:
        try:
            irf_l = res_low.irf(15)
            plot_irf_save(
                irf_l,
                f"IRF - {ticker} BEFORE - Low Sentiment",
                os.path.join(out_dir, f"{ticker}_BEFORE_IRF_low.png"),
                orth=False
            )
        except Exception as e:
            print("IRF Low failed:", e)

    if res_high is not None:
        try:
            irf_h = res_high.irf(15)
            plot_irf_save(
                irf_h,
                f"IRF - {ticker} BEFORE - High Sentiment",
                os.path.join(out_dir, f"{ticker}_BEFORE_IRF_high.png"),
                orth=False
            )
        except Exception as e:
            print("IRF High failed:", e)

# ===== run all =====
if __name__ == "__main__":
    files = glob.glob(os.path.join(folder_before, "*.xlsx"))
    if not files:
        print("Không thấy .xlsx trong:", folder_before)

    for f in files:
        ticker = os.path.splitext(os.path.basename(f))[0]
        run_before(f, ticker)



[Applied differences]
  ret: diff 0 time(s)
  score: diff 1 time(s)

[Stationarity checks]
  var        ADF_p  KPSS_p   ADF_stat  KPSS_stat  ADF_lags  KPSS_lags  nobs
  ret 3.949181e-20     0.1 -11.097081   0.327076         4         11  1030
score 9.922521e-21     0.1 -11.352373   0.201002        18        242  1016

=== AMD (BEFORE) ===
Obs Low: 610 | Obs High: 424 | Threshold≈ 0.001

[Low regime] lag = 8
  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 02, Oct, 2025
Time:                     00:56:54
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                 -0.0274511
Nobs:                     602.000    HQIC:                 -0.179241
Log likelihood:          -1591.33    FPE:                   0.758858
AIC:                    -0.275971    Det(Omega_mle):        0.717748
--------------------------------------------------------------------